In [1]:
import numpy as np
import pandas as pd
import random
from collections import deque

random.seed(261)

In [2]:
def initialise_cycle(size):
    state = np.zeros((size, size), dtype=int)

    for i in range(size):
        if random.random() > 0.5:
            state[i, (i+1)%size] = 1
        else:
            state[(i+1)%size, i] = 1
    
    return state

In [3]:
def get_gen_score_vec(state):
    return np.sum(state, axis=1)

In [4]:
def sample_transition(transition_matrix):
    size = transition_matrix.shape[0]
    idxs = np.array([(i, j) for i in range(size) for j in range(size)], dtype="i,i").astype(object)
    return np.random.choice(idxs, p=np.ravel(transition_matrix))

In [5]:
def bfs(state, source, target):
    visited = [target]
    queue = deque([[target]])

    while queue:
        cur_node = queue.popleft()
        # print(cur_node)
        last_visited = cur_node[-1]
        if last_visited == source:
            return cur_node

        for neighbour in np.nonzero(np.atleast_1d(np.asarray(state[last_visited] > 0)))[0]:
            if neighbour not in visited:
                visited.append(neighbour)
                queue.append(cur_node + [neighbour])
                # print(queue)
    
    return None

In [6]:
def do_transaction(state, solution):
    if len(solution) > 1:
        for (s, t) in zip(solution[:-1], solution[1:]):
            state[s, t] -= 1
            state[t, s] += 1
    return state

In [7]:
transition_matrix = np.array([[0, 1/3, 0], [0, 0, 1/3], [1/3, 0, 0]])

In [8]:
NUM_SIMS = 100
NUM_ITERS = 10000

In [9]:
NUM_NODES = 3

In [10]:
all_sims = []

for i in range(NUM_SIMS):
    if (i+1)%50 == 0:
        print(f"Sim {(i+1):03d}")

    state = initialise_cycle(NUM_NODES)

    all_states = [state.copy()]
    for i in range(NUM_ITERS):
        # if (i+1)%1000 == 0:
        #     print(f"Iter {(i+1):05d}")
        (source, target) = sample_transition(transition_matrix)
        soln = bfs(state, source, target)
        if soln is None:
            all_states.append(state.copy())
        else:
            new_state = do_transaction(state, soln)
            all_states.append(new_state.copy())
            state = new_state
    
    all_sims.append(all_states)

Sim 050
Sim 100


In [11]:
vec_list = ['[1 1 1]',
            '[0 2 1]',
            '[2 1 0]',
            '[1 0 2]',
            '[2 0 1]',
            '[0 1 2]',
            '[1 2 0]']

In [12]:
def get_cumprops(states):
    gen_score_vecs = np.array([np.array_str(get_gen_score_vec(s)) for s in states])
    vals = [(gen_score_vecs == v).astype(int) for v in vec_list]
    cum_vals = [np.divide(np.cumsum(v), (range(1,NUM_ITERS+2))) for v in vals]
    return cum_vals

In [13]:
def make_long(cumprop, i):
    df = pd.DataFrame(dict(zip(vec_list, cumprop)))
    df["iter"] = df.index
    long = pd.melt(df, id_vars="iter")
    long["sim"] = i
    return long

In [14]:
all_freqs = [get_cumprops(states) for states in all_sims]

In [15]:
all_longs = pd.concat([make_long(cumprop, i) for (i, cumprop) in enumerate(all_freqs)])

In [16]:
all_longs.to_csv("all_longs.csv")

In [17]:
def make_transition_matrix(size):
    t_matrix = np.random.random((size, size))
    np.fill_diagonal(t_matrix, 0)
    return t_matrix / np.sum(t_matrix)

In [18]:
def make_p_matrix(transition_matrix):
    tm = transition_matrix
    p_matrix = np.array([
        [0, tm[1,0], tm[0,2], tm[2,1], tm[0,1], tm[1,2], tm[2,0]],
        [tm[0,1], tm[1,0]+tm[1,2]+tm[2,0], 0, 0, 0, tm[2,1], tm[0,2]],
        [tm[2,0], 0, tm[0,1]+tm[0,2]+tm[1,2], 0, tm[2,1], 0, tm[1,0]],
        [tm[1,2], 0, 0, tm[0,1]+tm[2,0]+tm[2,1], tm[0,2], tm[1,0], 0],
        [tm[1,0], 0, tm[1,2], tm[2,0], tm[0,1]+tm[0,2]+tm[2,1], 0, 0],
        [tm[0,2], tm[1,2], 0, tm[0,1], 0, tm[1,0]+tm[2,0]+tm[2,1], 0],
        [tm[2,1], tm[2,0], tm[0,1], 0, 0, 0, tm[0,2]+tm[1,2]+tm[1,0]]
    ])
    return p_matrix

In [19]:
pm = make_p_matrix(make_transition_matrix(3))

In [20]:
def get_distrib(pm):
    eigval, eigvec = np.linalg.eig(pm.T)
    unit_eig = abs(eigval - 1.) < 1e-14 # account for floating point errors
    if (np.sum(unit_eig) != 1):
        return None
    vec = eigvec[:, unit_eig].real
    return vec / np.sum(vec)

In [21]:
NUM_SIMS = 200

In [22]:
all_sims = []
all_trans = []

for i in range(NUM_SIMS):
    if (i+1)%50 == 0:
        print(f"Sim {(i+1):03d}")

    state = initialise_cycle(NUM_NODES)
    transition_matrix = make_transition_matrix(NUM_NODES)

    all_states = [state.copy()]
    all_trans.append(transition_matrix.copy())
    for i in range(NUM_ITERS):
        # if (i+1)%1000 == 0:
        #     print(f"Iter {(i+1):05d}")
        (source, target) = sample_transition(transition_matrix)
        soln = bfs(state, source, target)
        if soln is None:
            all_states.append(state.copy())
        else:
            new_state = do_transaction(state, soln)
            all_states.append(new_state.copy())
            state = new_state
    
    all_sims.append(all_states)

Sim 050
Sim 100
Sim 150
Sim 200


In [23]:
all_freqs = [get_cumprops(states) for states in all_sims]
all_longs = pd.concat([make_long(cumprop, i) for (i, cumprop) in enumerate(all_freqs)])

In [24]:
final_distribs = all_longs.loc[all_longs["iter"] == 10000].copy()

In [25]:
distribs = [get_distrib(make_p_matrix(tm)) for tm in all_trans]

In [26]:
final_distribs["estimated"] = np.concatenate(distribs)

In [27]:
final_distribs.to_csv("final_distribs.csv")

In [28]:
def calc_symmetry(t_matrix):
    t_sym = (t_matrix + t_matrix.T) / 2
    t_anti = (t_matrix - t_matrix.T) / 2
    norm_sym = np.linalg.norm(t_sym, ord="fro")
    norm_anti = np.linalg.norm(t_anti, ord="fro")
    return (norm_sym - norm_anti) / (norm_sym + norm_anti)


In [29]:
transition_matrix

array([[0.        , 0.1556791 , 0.15195592],
       [0.16529819, 0.        , 0.26518694],
       [0.01418463, 0.24769522, 0.        ]])

In [30]:
symmetries = [calc_symmetry(tm) for tm in all_trans]

In [31]:
symmetries

[0.31506798259528,
 0.4509800077088282,
 0.2989147593531458,
 0.4677220468275161,
 0.6731969304581639,
 0.6333141409263889,
 0.2976908151751545,
 0.38828657130323896,
 0.8287517095242319,
 0.4618166479933317,
 0.5540439984321982,
 0.48765708794438795,
 0.1622784277861656,
 0.6296231567738102,
 0.10792039442991402,
 0.49479091670962805,
 0.27743161599326005,
 0.3880933215931853,
 0.17554525585745465,
 0.2578385838231851,
 0.8370923744367464,
 0.5007865814901167,
 0.646404422792171,
 0.3331155656215466,
 0.14337816909718248,
 0.6195436352935871,
 0.4453402705047659,
 0.9149349398996053,
 0.682835997932818,
 0.43310047507613,
 0.23877332313264948,
 0.4585166113701104,
 0.6988866917908874,
 0.32155115143990404,
 0.48739876663405407,
 0.449457800290104,
 0.05222145651588973,
 0.4612986117114183,
 0.30477097883187565,
 0.3211538083751845,
 0.44038531379068097,
 0.6062135441191678,
 0.3882142365315734,
 0.26127013682180106,
 0.43718650867808817,
 0.3843214341018378,
 0.32759170842219537,
 0.7

In [35]:
rmse = final_distribs.groupby("sim").apply(lambda x: np.sqrt(np.mean((x["value"]-x["estimated"])**2)))

In [39]:
def calc_entropy(distrib):
    return -1 * np.sum(distrib * np.log2(distrib))

In [42]:
sim_metrics = final_distribs.groupby("sim").agg({'value': calc_entropy, 'estimated': calc_entropy})

In [43]:
sim_metrics["symmetry"] = symmetries

In [ ]:
sim_metrics.to_csv("sim_metrics.csv")